# Dependencies

In [1]:
!pip install cheshire_cat_api

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 1.8 MB/s eta 0:00:00


# Code

In [2]:
import re

def split_and_save(file_path, output_folder, delimiter, prefix, allowed):
    # Open and read the content of the source file
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    content = clean_text(allowed, content)

    # Regular expression to match "Art. X" where X is a number
    pattern = r'(' + delimiter + r')'

    # Find all occurrences of the pattern
    articles = re.split(pattern, content)

    # The first item in 'articles' might not start with "Art. X" and can be an introduction or preface
    # If it doesn't match our pattern, we'll ignore it in file writing
    if not re.match(pattern, articles[0]):
        articles = articles[1:]

    # Iterate over the articles to write them to separate files
    for i in range(1, len(articles), 2):
        article_number = articles[i].replace('\n', ' ').replace(':', ' ').split(" ")[0]  # Extract the number X from "Art. X"
        file_name = f'art-{article_number}.txt'  # Create the file name
        # Write the content to the file
        with open(f"{output_folder}/{prefix}_" + file_name, 'w', encoding='utf-8') as file:
            file.write(f"{delimiter}" + articles[i])


def save_result(res, file_path):
  with open(file_path, 'w', encoding='utf-8') as file:
    file.write(res[0])


def split_files_in_folder(folder_path, output_folder, delimiter, allowed):

  import os
  files = os.listdir(folder_path)

  for file_name in files:
    if file_name.endswith(".txt"):
      file_path = os.path.join(folder_path, file_name)
      split_and_save(file_path, output_folder, delimiter, file_name[:-4], allowed)

def clean_text(allowed, content):
  if not allowed:
    return content
  return "\n".join([x for x in content.split("\n") if any([x.startswith(y) for y in allowed])])

In [3]:
import time
import json
import cheshire_cat_api as ccat

content = ""

def on_open():
    # This is triggered when the connection is opened
    print("Connection opened!")

def on_message(message: str):
    # This is triggered when a new message arrives
    # and grabs the message
    # print(message)
    global content

    response = json.loads(message)
    if response["type"] != "chat_token":
      print(response)
    if response["type"] == "chat":
      content = response

def on_error(exception: Exception):
    # This is triggered when a WebSocket error is raised
    global content
    print(str(exception))
    content = str(exception)

def on_close(status_code: int, message: str):
    # This is triggered when the connection is closed
    print(f"Connection closed!")



def connect(user_id):
  # Connection settings with default values
  config = ccat.Config(
      base_url="WEBSITE",
      port=port,
      user_id=user_id,
      auth_key="",
      secure_connection=False
  )

  # Cat Client
  cat_client = ccat.CatClient(
      config=config,
      on_open=on_open,
      on_close=on_close,
      on_message=on_message,
      on_error=on_error
  )

  # Connect to the WebSocket API
  cat_client.connect_ws()

  while not cat_client.is_ws_connected:
      time.sleep(1)

  return cat_client

def disconnect(cat_client):
  cat_client.close()


def ask_gpt(cat_client, prompt):
  global content

  content = ""
  cat_client.send(message=prompt)

  while content == "":
    True

  return content


def prompt(user_id, prompt):
  client = connect(user_id)
  res = ask_gpt(client, prompt)
  disconnect(client)
  return res["content"], [( x["metadata"]["source"], x["score"], x["page_content"]) for x in res["why"]["memory"]["declarative"]]


def print_response(res):
  print("Output:\n")
  print(res[0])
  print("\n\nMemories:\n\n")
  for x in res[1]:
    print(f"{x[0]} - {x[1]}\n{x[2]}\n\n")


def clean_history(user):
  import requests

  url = "http://WEBSITE:port/memory/conversation_history/"

  payload = {}
  headers = {
    'Accept': 'application/json',
    'user_id': user
  }

  response = requests.request("DELETE", url, headers=headers, data=payload)
  print(response.text)

def get_history(user):
  import requests

  url = "http://WEBSITE:port/memory/conversation_history/"

  payload = {}
  headers = {
    'Accept': 'application/json',
    'user_id': user
  }

  response = requests.request("GET", url, headers=headers, data=payload)
  print(response.text)

def upload_memory(user, chunk_size, file_name, file_path):
  import http.client
  import mimetypes
  from codecs import encode
  import time

  conn = http.client.HTTPConnection("WEBSITE", port)
  dataList = []
  boundary = 'boundary'
  dataList.append(encode('--' + boundary))
  dataList.append(encode('Content-Disposition: form-data; name=file; filename={0}'.format(file_name)))

  fileType = mimetypes.guess_type(file_path)[0] or 'application/octet-stream'
  dataList.append(encode('Content-Type: {}'.format(fileType)))
  dataList.append(encode(''))

  with open(file_path, 'rb') as f:
    dataList.append(f.read())
  dataList.append(encode('--' + boundary))
  dataList.append(encode('Content-Disposition: form-data; name=chunk_size;'))

  dataList.append(encode('Content-Type: {}'.format('text/plain')))
  dataList.append(encode(''))

  dataList.append(encode(str(chunk_size)))
  dataList.append(encode('--' + boundary))
  dataList.append(encode('Content-Disposition: form-data; name=chunk_overlap;'))

  dataList.append(encode('Content-Type: {}'.format('text/plain')))
  dataList.append(encode(''))

  dataList.append(encode("100"))
  dataList.append(encode('--'+boundary+'--'))
  dataList.append(encode(''))
  body = b'\r\n'.join(dataList)
  payload = body
  headers = {
    'Content-type': 'multipart/form-data; boundary={}'.format(boundary),
    'Accept': 'application/json',
    'user_id': user
  }
  conn.request("POST", "/rabbithole/", payload, headers)
  res = conn.getresponse()
  data = res.read()
  print(data.decode("utf-8"))

def upload_all_memories_folder(user, chunk_size, folder):
  import os
  files = os.listdir(folder)

  for file_name in files:
    if file_name.endswith(".txt"):
      file_path = os.path.join(folder, file_name)
      upload_memory(user, chunk_size, file_name, file_path)
      time.sleep(1)


def summarise_in_folder(user, folder, target_folder):
  import os
  files = os.listdir(folder)

  for file_name in files:
    if file_name.endswith(".txt"):
      file_path = os.path.join(folder, file_name)
      prompt_summary_r = prompt_summary(file_path)
      clean_history(user)
      summary = prompt(user, prompt_summary_r)
      save_result(summary, f"{target_folder}/{file_name}")


def set_declarative_memories(n, t):
  import http.client
  import json

  conn = http.client.HTTPConnection("WEBSITE", port)

  payload = json.dumps({
    "language": "English",
    "only_local_responses": False,
    "prompt_prefix": "",
    "disable_episodic_memories": True,
    "disable_declarative_memories": False,
    "disable_procedural_memories": False,
    "number_of_declarative_items": n,
    "number_of_episodic_items": 30,
    "declarative_threshold": t,
    "episodic_threshold": 0.8
  })

  headers = {
    'Content-Type': 'application/json',
    'Accept': 'application/json'
  }
  conn.request("PUT", "/plugins/settings/cc_prompt_settings", payload, headers)
  res = conn.getresponse()
  data = res.read()
  print(data.decode("utf-8"))

def set_model_temperature(t):
  import http.client
  import json

  conn = http.client.HTTPConnection("WEBSITE", port)
  payload = json.dumps({
    "openai_api_key": "KEY",
    "model_name": "gpt-4-turbo-preview",
    "temperature": t,
    "streaming": True
  })
  headers = {
    'Content-Type': 'application/json',
    'Accept': 'application/json'
  }
  conn.request("PUT", "/llm/settings/LLMOpenAIChatConfig", payload, headers)
  res = conn.getresponse()
  data = res.read()
  print(data.decode("utf-8"))

# Data

In [4]:
prolog_tree_dutch = """
  directive_2010_64_nl - art32a_1

  Article 32a(1) Dutch Code of Criminal Procedure
  Option: essentialDocument

  Explanation:

  has_right(right_to_translation, nl, art32a_1, mario, essentialDocument)
  has_right(art32a_1, mario, right_to_translation, essentialDocument)
  person_status(mario, suspect) [FACT]
  person_request_submitted(mario, essential_document) [FACT]
  Auxiliaries:

  art1_4 - cost - state

  Article 1(4) The Criminal Cases Fees Act
  Explanation:

  auxiliary_right(art1_4, art32a_1, mario, cost, state)
  auxiliary_right(art1_4, mario, cost, state)
"""

prolog_tree_directive = """
  directive_2010_64 - art3_1

  Article 3
  Option: essentialDocument

  Explanation:

  has_right(right_to_translation, dir, art3_1, mario, essentialDocument)
    has_right(art3_1, mario, right_to_translation, essentialDocument)
      proceeding_language(mario, dutch) [FACT]
      essential_document(art3_3, mario, documents)
        authority_decision(mario, essential_document) [FACT]
      not(person_understands(mario, dutch))

  Auxiliaries:

  art4 - cost - state

  Article 4
  Explanation:

  auxiliary_right(art4, art3_1, mario, cost, state)
      auxiliary_right(art4, mario, cost, state)

  Properties:

  art3_7 - form - oral

  Article 3.7
  Explanation:

  right_property(art3_7, art3_1, mario, form, oral)
      right_property(art3_7, mario, form, oral)
          not(proceeding_event(mario, prejudice_fairness))
"""

# Prompts

In [5]:

prompt_0_1 = """
You have been provided a Prolog inference tree using a legal norm in a specific case (Prolog Tree).
Provide the following info according to the given structure:

Summary: simplified text of the legal norm. Use everyday langaguage with a serious register;
What Rights do You Have: all main rights referenced by a corresponding positive Prolog statement;
Why do You Have Them: inference steps and reasoning that led to the the rights. Use all the Prolog terms in the explanation explicitly referencing the original Prolog when needed.

Use enumerations in the 'What Rights do You Have' and 'Why do You Have Them' sections if needed.
"""

def prompt_1(res_dir, res_national):

  _prompt_1_1 = """
    You have received two legal sources. Compare them according to the following steps:
    1 - Make a summary of the legal case referencing the Prolog facts in the 'Why do You Have Them' section. Make one summary for each legal source;
    2 - Compare which Prolog facts are in common between the two legal sources, and which are the source of differences;
    3 - analyse consequences deriving from step 1 and 2. Explain your reasoning;
    4 - Which legal source favours the individual?

    THE ANALYSIS MUST FOCUS ON THE CASE AND NOT ON THE GENERAL LEGAL SOURCES
  """

  def _memories(res, n):
    return  "\n\n".join([x[2] for x in res[1][:n]])

  return f"""
    Legal Source:

    {res_dir[0]}

    Legal Source:

    {res_national[0]}

    Context:

    {_memories(res_dir, 5)}

    {_memories(res_national, 5)}


  """ + _prompt_1_1


def prompt_2_all_identifiers(res):

  _prompt_2_4 = """
    You have received a legal analysis of a case.
    You will receive a list of case parts. You can find the case identifier looking at the label (extracted from X), with X the identifier.
    You must ignore the _art part in the identifier.

    Identify court cases according to the following steps:
    1 - Make a list of all the new court cases;
    2 - For each court case provide a list of the main factual premises
  """

  return res[0] + "\n\n" + _prompt_2_4


def prompt_2_relevant_identifiers(res, list):

  _prompt_2_4 = """
    You have received a legal analysis of a case and a list of court cases with their main factual elements and norms.

    Identify court cases according to the following steps:
    1 - Focus on the relevant factual elements and norms in the original case;
    2 - Rank the court cases according to the highest number of norms and factual premises in common;
    3 - Discard the least relevant court cases;
    4 - Return the list of cases identifiers.
  """

  return res[0] + "\n\n" + list[0] + "\n\n" + _prompt_2_4

def prompt_2_analysis(res):

  _prompt_1_1 = """
    You have received two legal sources: an analysis of legal dispute and parts of a court case. Compare them according to the following steps:
    1 - Make a summary of the legal court case;
    2 - Compare which facts are in common between the two legal sources, and which are the source of differences;
    3 - analyse consequences deriving from step 1 and 2. Explain your reasoning;

    THE ANALYSIS MUST FOCUS ON THE CASE AND NOT ON THE GENERAL LEGAL SOURCES
  """

  return "ANALYSIS: \n\n" + res[0] + "\n\n" + _prompt_1_1


def prompt_summary(identifier):

  _prompt_summary_1 = """
    You have received the full text of a court case. Create a summary according to the following steps:
    1 - Identify the applicable legal norms;
    2 - Provide a description of the premises (factual elements) of each legal norm.

    The summary should be in English, should not exceed 2000 characters and should follow the following structure:

    Norm 1 : Norm 1 complete reference
    Description 1 : Description of Norm 1 and its factual elements
    ...
    Norm 1 : Norm N complete reference
    Description N : Description of Norm N and its factual elements

  """

  with open(identifier, 'r', encoding="utf-8") as f:
    txt = f.read()
    return txt + "\n\n" + _prompt_summary_1

# Task 1

In [12]:
clean_history("directive-art")
clean_history("dutch-artt")

set_declarative_memories(10, 0.4)
dutch = prompt("dutch-artt", f"Prolog Tree: \n\n {prolog_tree_dutch} \n\n {prompt_0_1}")
directive = prompt("directive-art", f"Prolog Tree: \n\n {prolog_tree_directive} \n\n {prompt_0_1}")

{"deleted":true}
{"deleted":true}
{"name":"cc_prompt_settings","value":{"language":"English","only_local_responses":false,"prompt_prefix":"","disable_episodic_memories":true,"disable_declarative_memories":false,"disable_procedural_memories":false,"number_of_declarative_items":10,"number_of_episodic_items":30,"declarative_threshold":0.4,"episodic_threshold":0.8}}
Connection opened!
{'type': 'chat', 'user_id': 'directive-art', 'content': "### Summary\n\nIn simpler terms, the legal norm discussed here is about the right to have essential documents translated for individuals who are involved in criminal proceedings but do not understand the language those proceedings are conducted in. This right ensures that such individuals can defend themselves properly and that the fairness of the proceedings is maintained. The law states that the costs associated with this translation will be covered by the state, and in certain situations, an oral translation may be provided instead of a written one, 

In [9]:
print_response(dutch)

Output:

Summary:
If you are a suspect in a legal case in the Netherlands and you don't fully understand Dutch, you have the right to ask for important documents related to your case to be translated into a language you understand. This is to ensure you can properly defend yourself. Additionally, if there are any costs involved in getting these translations, the state will cover them for you.

What Rights do You Have:
1. The right to request the translation of essential documents for your defense into a language you understand.
2. The right to have the costs of translation covered by the state.

Why do You Have Them:
1. You are identified as a suspect (person_status(mario, suspect) [FACT]), which grants you certain rights under the Dutch Code of Criminal Procedure.
2. You have submitted a request for essential documents to be translated (person_request_submitted(mario, essential_document) [FACT]), as allowed by Article 32a(1) of the Dutch Code of Criminal Procedure. This article specif

In [13]:
print_response(directive)

Output:

### Summary

In simpler terms, the legal norm discussed here is about the right to have essential documents translated for individuals who are involved in criminal proceedings but do not understand the language those proceedings are conducted in. This right ensures that such individuals can defend themselves properly and that the fairness of the proceedings is maintained. The law states that the costs associated with this translation will be covered by the state, and in certain situations, an oral translation may be provided instead of a written one, as long as it doesn't affect the fairness of the trial. This comes from Directive 2010/64/EU, which aims to make sure that people have the necessary linguistic support during criminal proceedings.

### What Rights do You Have

1. **Right to Translation of Essential Documents**: You have the right to get a written translation of all documents that are crucial for you to exercise your defense and to ensure the fairness of the procee

# Task 2

In [ ]:
prompt_1_r = prompt_1(directive, dutch)
clean_history("clean")
set_declarative_memories(10, 0.4)
comparison = prompt("clean", prompt_1_r)

{"deleted":true}
{"name":"cc_prompt_settings","value":{"language":"English","only_local_responses":false,"prompt_prefix":"","disable_episodic_memories":true,"disable_declarative_memories":false,"disable_procedural_memories":false,"number_of_declarative_items":10,"number_of_episodic_items":30,"declarative_threshold":0.4,"episodic_threshold":0.8}}
Connection opened!
{'type': 'chat', 'user_id': 'clean', 'content': "### Step 1: Summary of the Legal Case Based on Prolog Facts\n\n#### European Union Directive Summary:\nMario, identified as a suspect in a criminal proceeding, does not understand the language used in the trial. Under the European Union directive, he is granted the right to have essential documents translated, ensuring he can exercise his right of defense and safeguard the fairness of the proceedings. This includes the right to challenge decisions regarding the necessity and quality of translations and to have the costs covered by the state. The directive emphasizes the importa

In [ ]:
print_response(comparison)

Output:

### Step 1: Summary of the Legal Case Based on Prolog Facts

#### European Union Directive Summary:
Mario, identified as a suspect in a criminal proceeding, does not understand the language used in the trial. Under the European Union directive, he is granted the right to have essential documents translated, ensuring he can exercise his right of defense and safeguard the fairness of the proceedings. This includes the right to challenge decisions regarding the necessity and quality of translations and to have the costs covered by the state. The directive emphasizes the importance of written translations but allows for oral summaries under certain conditions, ensuring the proceedings' fairness is not prejudiced.

#### Dutch Code of Criminal Procedure Summary:
According to the Dutch Code of Criminal Procedure, Mario, as a suspect who does not fully understand Dutch, has the right to request the translation of essential documents into a language he understands. This is crucial for 

# Task 3

In [ ]:
print_response(similar_cases_id)

Output:

To identify similar court cases based on the provided legal analysis, we'll follow the outlined steps focusing on the relevant factual elements and norms in the original case. The key elements include the right to interpretation and translation in criminal proceedings, the necessity and quality of translations, the coverage of costs by the state, and the procedural aspects related to requesting translations.

### Step 1: Focus on Relevant Factual Elements and Norms

- **Right to interpretation and translation in criminal proceedings**
- **Necessity and quality of translations**
- **State covers the costs of translation**
- **Procedural aspects of requesting translations**

### Step 2: List of Common Elements for Each Case

1. **sleutjes_case_art-1.txt**
   - Right to interpretation and translation
   - Coverage of costs by the state
   - Necessity and quality of translations

2. **Case_C_216_14_art-1.txt**
   - Right to interpretation and translation
   - Coverage of costs by 

In [ ]:
prompt_2_alt_rr = prompt_2_only_identifiers(comparison)
clean_history("cases-sum-art-03-final")
set_model_temperature(0.0)
set_declarative_memories(20, 0.4)
similar_cases_id_2 = prompt("cases-sum-art-03-final", prompt_2_alt_rr)
set_model_temperature(0.0)

In [ ]:
print_response(similar_cases_id_2)

Output:

Based on the provided instructions and the context of the documents, here is the analysis:

### 1 - List of All New Court Cases:

- sleutjes_case
- Barca_290_20
- Case_C_216_14
- hermi_vs_italy
- Cassazione_2019_8320
- bohkonko_vs_georgia
- Cassazione_2019_14413

### 2 - For Each Court Case, A List of All the Factual Elements and Norms:

- **sleutjes_case**
  - Factual Elements: Not explicitly detailed.
  - Norms: Directive 2010/64/EU, European Convention on Human Rights (ECHR), Article 6.

- **Barca_290_20**
  - Factual Elements: Spanish court's initial decision to deny the search and seizure request without consulting Dutch authorities.
  - Norms: Law 23/2014, Article 206, paragraph 4; Law 23/2014, Article 205 and subsequent articles; Article 206.

- **Case_C_216_14**
  - Factual Elements: Individuals involved in criminal proceedings who do not understand or speak the language used.
  - Norms: Directive 2010/64/EU.

- **hermi_vs_italy**
  - Factual Elements: The applicant's 

In [ ]:
prompt_2_alt_all_id = prompt_2_all_identifiers(comparison)
clean_history("cases-sum-art-03-final")
set_model_temperature(0.0)
set_declarative_memories(15, 0.4)
set_model_temperature(0.0)
all_cases_id_2 = prompt("cases-sum-art-03-final", prompt_2_alt_all_id)

In [ ]:
print_response(all_cases_id_2)

Output:

### 1 - List of All New Court Cases:

1. sleutjes_case
2. Case_C_216_14
3. hermi_vs_italy
4. Barca_290_20
5. Cassazione_2019_8320
6. bohkonko_vs_georgia
7. Cassazione_2022_48330

### 2 - Main Factual Premises for Each Court Case:

#### sleutjes_case:
- The directive ensures the right to interpretation and translation for individuals accused or suspected of a criminal offense within the EU.
- It mandates that essential documents in criminal proceedings be translated for those who do not understand the language of the procedure.
- Article 6 of the ECHR establishes the right to a fair trial, including the right for the accused to understand and be understood in judicial proceedings.

#### Case_C_216_14:
- Establishes the right to interpretation and translation for individuals involved in criminal proceedings who do not understand or speak the language used in those proceedings.
- Applies from the time individuals are made aware they are suspected or accused of a crime until the c

In [ ]:
prompt_2_alt_rel_id = prompt_2_relevant_identifiers(comparison, all_cases_id_2)
clean_history("nnnnnn")
set_declarative_memories(20, 0.4)
all_cases_id_3 = prompt("nnnnnn", prompt_2_alt_rel_id)

{"deleted":true}
{"name":"cc_prompt_settings","value":{"language":"English","only_local_responses":false,"prompt_prefix":"","disable_episodic_memories":true,"disable_declarative_memories":false,"disable_procedural_memories":false,"number_of_declarative_items":20,"number_of_episodic_items":30,"declarative_threshold":0.4,"episodic_threshold":0.8}}
Connection opened!
{'type': 'chat', 'user_id': 'nnnnnn', 'content': "Given the original case's focus on the right to translation and interpretation for individuals who do not understand the language used in criminal proceedings, we should prioritize court cases that directly address these issues. The relevant factual elements and norms include the right to a fair trial, the necessity of understanding and being understood in judicial proceedings, and the procedural aspects related to translation and interpretation rights under EU directives and the European Convention on Human Rights (ECHR).\n\n1. **Relevant Factual Elements and Norms:**\n   - R

In [ ]:
print_response(all_cases_id_3)

Output:

Given the original case's focus on the right to translation and interpretation for individuals who do not understand the language used in criminal proceedings, we should prioritize court cases that directly address these issues. The relevant factual elements and norms include the right to a fair trial, the necessity of understanding and being understood in judicial proceedings, and the procedural aspects related to translation and interpretation rights under EU directives and the European Convention on Human Rights (ECHR).

1. **Relevant Factual Elements and Norms:**
   - Right to interpretation and translation in criminal proceedings.
   - Right to a fair trial, including understanding and being understood.
   - Procedural requirements for requesting translations and the state's obligation to cover costs.

2. **Ranking Court Cases:**
   - **sleutjes_case:** Directly addresses the directive on interpretation and translation rights in the EU, closely aligned with the original c

In [ ]:
an_c = prompt_2_analysis(comparison, all_cases_id_3)
clean_history("nnnnnn")
set_declarative_memories(20, 0.4)
an_c = prompt("nnnnnn", prompt_2_analysis)

In [ ]:
print_response(an_c)

Output:

### Summary of the Legal Court Case

The court case revolves around the interpretation and application of Directive 2010/64/EU of the European Parliament and the Council of 20 October 2010 on the right to interpretation and translation in criminal proceedings, particularly within the context of German law. The central issue is whether a "Strafbefehl" (criminal order) should be considered an "essential document" that must be translated for a defendant who does not understand German. The Landgericht Aachen (Regional Court Aachen, Germany) sought a preliminary ruling on this matter, emphasizing the importance of ensuring fair proceedings by allowing the defendant to be fully informed and able to exercise their right of defense through adequate translation.

The Advocate General and the court considered various aspects, including the quality of translations provided, the defendant's rights to challenge the necessity and quality of translations, and the implications of not translat